In [1]:
%cd /home/davidhenao695/Python/Backend/FastAPI/video_membership

/home/davidhenao695/Python/Backend/FastAPI/video_membership


In [2]:
!pwd # path

/home/davidhenao695/Python/Backend/FastAPI/video_membership


In [3]:
from app.db.models import Video, Playlist
from sqlmodel import select
from app.db.database import async_session
from sqlmodel.ext.asyncio.session import AsyncSession
from pydantic import BaseModel, Field, field_validator, model_validator
from uuid import UUID
from typing import Optional

session: AsyncSession  =  async_session()

In [4]:
from app.db.schemas_indexing import *
from app.indexing.client import get_index, update_index

In [5]:
statement = select (Video)
video_db = await session.exec(statement)
video = video_db.first()
video_dict = video.__dict__

2024-07-04 18:35:31,999 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-07-04 18:35:32,001 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-04 18:35:33,637 INFO sqlalchemy.engine.Engine select current_schema()
2024-07-04 18:35:33,639 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-04 18:35:34,847 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-07-04 18:35:34,849 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-04 18:35:35,738 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-04 18:35:35,755 INFO sqlalchemy.engine.Engine SELECT videos.db_id, videos.host_service, videos.host_id, videos.url, videos.title, videos.owner_id 
FROM videos
2024-07-04 18:35:35,756 INFO sqlalchemy.engine.Engine [generated in 0.00160s] ()


In [6]:
class VideoIndex(BaseModel):
    objectID : str = Field(alias='host_id')
    title : Optional[str]
    path : str = Field(alias="host_id")
    objectType: str = "Video"

    @field_validator("path")
    @classmethod
    def set_path(cls, value):
        host_id = value
        return f"/videos/{host_id}"

index = VideoIndex(**video_dict)
index.dict()

{'objectID': 'VO8Bk03Xv90',
 'title': 'Obi-Wan - Obi-Wan vs Darth Vader - (HDR )',
 'path': '/videos/VO8Bk03Xv90',
 'objectType': 'Video'}

In [7]:
statement = select (Playlist)
playlist_db = await session.exec(statement)
playlist = playlist_db.first()
playlist_dict = playlist.__dict__
playlist_dict

2024-07-04 18:35:36,636 INFO sqlalchemy.engine.Engine SELECT playlist.db_id, playlist.owner_id, playlist.host_ids, playlist.updated, playlist.title 
FROM playlist
2024-07-04 18:35:36,651 INFO sqlalchemy.engine.Engine [generated in 0.01479s] ()


{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x7f42f7f2ccb0>,
 'updated': datetime.datetime(2024, 6, 28, 7, 39, 39, 415521, tzinfo=datetime.timezone.utc),
 'title': 'Hello world',
 'db_id': UUID('5afc9592-346a-11ef-a94b-00163e90c528'),
 'owner_id': UUID('dca2ebb8-33c6-11ef-af11-00163e90c528'),
 'host_ids': ['VO8Bk03Xv90', 'q9NuIVwYa9Q']}

In [8]:
class PlaylistIndex(BaseModel):
    objectID : UUID = Field(alias='db_id')
    title : Optional[str]
    path : str = Field(default="/")
    objectType: str = "Playlist"

    @model_validator(mode='after')
    def set_defaults(self):
        self.objectID = str(self.objectID)
        self.path = f"/playlist/{self.objectID}"
        return self

PlaylistIndex(**playlist_dict).dict()

/home/davidhenao695/Python/Backend/FastAPI/video_membership/venv/lib/python3.11/site-packages/pydantic/main.py:347: UserWarning: Pydantic serializer warnings:
  Expected `uuid` but got `str` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


{'objectID': '5afc9592-346a-11ef-a94b-00163e90c528',
 'title': 'Hello world',
 'path': '/playlist/5afc9592-346a-11ef-a94b-00163e90c528',
 'objectType': 'Playlist'}

In [9]:
statement = select (Video)
video_db = await session.exec(statement)
video_list = video_db.all()

video_dic = [x.as_dict() for x in video_list]
video_dataset = [VideoIndex(**x).dict() for x in video_dic]
#video_dataset

2024-07-04 18:35:37,293 INFO sqlalchemy.engine.Engine SELECT videos.db_id, videos.host_service, videos.host_id, videos.url, videos.title, videos.owner_id 
FROM videos
2024-07-04 18:35:37,299 INFO sqlalchemy.engine.Engine [cached since 1.545s ago] ()


In [10]:
statement = select (Playlist)
playlist_db = await session.exec(statement)
playlist_list = playlist_db.all()

playlist_dic = [x.as_dict() for x in playlist_list]
playlist_dataset = [PlaylistIndex(**x).dict() for x in playlist_dic]
#playlist_dataset

2024-07-04 18:35:41,718 INFO sqlalchemy.engine.Engine SELECT playlist.db_id, playlist.owner_id, playlist.host_ids, playlist.updated, playlist.title 
FROM playlist
2024-07-04 18:35:41,722 INFO sqlalchemy.engine.Engine [cached since 5.086s ago] ()


In [11]:
dataset = playlist_dataset + video_dataset
len (dataset)

8

In [12]:
index = get_index()
#dir(index)
idx_response = index.save_objects(dataset).wait()
#idx_response[0]['objectIDs']
#len(idx_response[0]['objectIDs'])
count = await update_index()
len(list(idx_response)[0]['objectIDs'])

2024-07-04 18:35:47,393 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-04 18:35:47,395 INFO sqlalchemy.engine.Engine SELECT videos.db_id, videos.host_service, videos.host_id, videos.url, videos.title, videos.owner_id 
FROM videos
2024-07-04 18:35:47,400 INFO sqlalchemy.engine.Engine [cached since 11.65s ago] ()
2024-07-04 18:35:49,951 INFO sqlalchemy.engine.Engine SELECT playlist.db_id, playlist.owner_id, playlist.host_ids, playlist.updated, playlist.title 
FROM playlist
2024-07-04 18:35:49,953 INFO sqlalchemy.engine.Engine [cached since 13.32s ago] ()


8

In [17]:
#index.search('hello')
dir(index)
result = index.search("")
#result.get("nbHits")
index.replace_all_objects(dataset).wait()
result

{'hits': [{'title': 'Acoustic France (Official Putumayo Version)',
   'path': '/videos/xxKjh_bLnQQ',
   'objectType': 'Video',
   'objectID': 'xxKjh_bLnQQ',
   '_highlightResult': {'title': {'value': 'Acoustic France (Official Putumayo Version)',
     'matchLevel': 'none',
     'matchedWords': []},
    'path': {'value': '/videos/xxKjh_bLnQQ',
     'matchLevel': 'none',
     'matchedWords': []},
    'objectType': {'value': 'Video',
     'matchLevel': 'none',
     'matchedWords': []}}},
  {'title': 'Los Espiritus - Gratitud Full Album',
   'path': '/videos/q9NuIVwYa9Q',
   'objectType': 'Video',
   'objectID': 'q9NuIVwYa9Q',
   '_highlightResult': {'title': {'value': 'Los Espiritus - Gratitud Full Album',
     'matchLevel': 'none',
     'matchedWords': []},
    'path': {'value': '/videos/q9NuIVwYa9Q',
     'matchLevel': 'none',
     'matchedWords': []},
    'objectType': {'value': 'Video',
     'matchLevel': 'none',
     'matchedWords': []}}},
  {'title': 'Italian Café (Official Putumayo